In [1]:
import numpy as np
import xarray as xr
import crocosi.gridop as gop

from itertools import permutations

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [2]:
# generate synthetic data
dims = {'time':3, 'z':5, 'y':20, 'x':10}
vmap = {'time':0., 'z':1., 'y':0., 'x':0.}

In [3]:
ndim = 3
P = [p for p in list(permutations(dims.keys(),ndim)) if 'z' in p]
#for p in P
p = P[3]

_coords = {d: np.arange(dims[d]) for d in p}
_coords.update(z_target1d=np.arange(.5,6))
ds = xr.Dataset(coords=_coords)

ds['v'] = sum([ds[d]*vmap[d] for d in p])
ds['z_v'] = ds['z'] + 0.*ds['v']
# the line above necessarily imposes a dimension order which may vary
# should loop around all potential dimension order for zv

ds['z_target'] = ds['z_target1d']
# need to vary number of dimensions and their order on the line above

In [4]:
ds

<xarray.Dataset>
Dimensions:     (time: 3, x: 10, z: 5, z_target1d: 6)
Coordinates:
  * time        (time) int64 0 1 2
  * x           (x) int64 0 1 2 3 4 5 6 7 8 9
  * z           (z) int64 0 1 2 3 4
  * z_target1d  (z_target1d) float64 0.5 1.5 2.5 3.5 4.5 5.5
Data variables:
    v           (time, x, z) float64 0.0 1.0 2.0 3.0 4.0 ... 0.0 1.0 2.0 3.0 4.0
    z_v         (z, time, x) float64 0.0 0.0 0.0 0.0 0.0 ... 4.0 4.0 4.0 4.0 4.0
    z_target    (z_target1d) float64 0.5 1.5 2.5 3.5 4.5 5.5

In [5]:
gop.interp2z(ds['z_target'], ds['v'], ds['z_v'])

ValueError: applied function returned data with unexpected number of dimensions: 3 vs 4, for dimensions ('z_target1d', 'time', 'x', 'z')